## Code for replicating gao et al research on VUA Sequence Model

In [1]:
# mount drive
from google.colab import drive
ROOT = '/content/drive'
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# add repo directory to path
import os
import sys
from os.path import join 
repo_dir = '/content/drive/MyDrive/Repos/metaphor-detection'
if repo_dir not in sys.path:
    sys.path.append(repo_dir)
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Repos/metaphor-detection']


In [3]:
# directories
# to download glove and elmo vectors see: notebooks/Download_large_data.ipynb
data_dir = repo_dir + '/resources/metaphor-in-context/data/'
glove_dir = repo_dir + '/resources/glove/'
elmo_dir = repo_dir + '/resources/elmo/'

In [13]:
# cd to working directory here if neccesary
%pwd

'/content/drive/MyDrive/Repos/metaphor-detection'

Gao code

In [11]:
# pip install requirements (takes a while)
!cd drive/MyDrive/Repos/metaphor-detection/; pip install -r gao-g-requirements.txt
!pip install --upgrade google-cloud-storage

/bin/bash: line 0: cd: drive/MyDrive/Repos/metaphor-detection/: No such file or directory
     |████████████████████████████████| 716 kB 8.7 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 592 kB 59.4 MB/s 
     |████████████████████████████████| 1.7 MB 52.9 MB/s 
     |████████████████████████████████| 125 kB 72.2 MB/s 
     |████████████████████████████████| 1.2 MB 48.1 MB/s 
     |████████████████████████████████| 240 kB 72.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.4 MB 28.2 MB/s 
     |████████████████████████████████| 12.1 MB 48.4 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 167 kB 70.5 MB/s 
     |████████████████████████████████| 132 kB 66.4 MB/s 
     |████████████████████████████████| 8.6 MB 

In [14]:
#@title
!pip install Ipython --upgrade

     |████████████████████████████████| 793 kB 6.6 MB/s 
     |████████████████████████████████| 380 kB 65.5 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: Ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.28 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.


In [15]:
%load_ext autoreload
%autoreload 2

In [16]:
from core.gao_files.sequence.util import get_num_lines, get_pos2idx_idx2pos, index_sequence, get_vocab, embed_indexed_sequence, \
    get_word2idx_idx2word, get_embedding_matrix, write_predictions, get_performance_VUAverb_val, \
    get_performance_VUAverb_test, get_performance_VUA_test
from core.gao_files.sequence.util import TextDatasetWithGloveElmoSuffix as TextDataset
from core.gao_files.sequence.util import evaluate
from core.gao_files.sequence.model import RNNSequenceModel

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

import csv
import h5py
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import ast

In [17]:
print("PyTorch version:")
print(torch.__version__)
print("GPU Detected:")
print(torch.cuda.is_available())
using_GPU = torch.cuda.is_available()

PyTorch version:
1.10.0+cu111
GPU Detected:
True


In [18]:
"""
1. Data pre-processing
"""
'''
1.1 VUA
get raw dataset as a list:
  Each element is a triple:
    a sentence: string
    a list of labels: 
    a list of pos: 
'''
pos_set = set()
raw_train_vua = []
with open(data_dir + 'VUAsequence/VUA_seq_formatted_train.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert (len(pos_seq) == len(label_seq))
        assert (len(line[2].split()) == len(pos_seq))
        raw_train_vua.append([line[2], label_seq, pos_seq])
        pos_set.update(pos_seq)

raw_val_vua = []
with open(data_dir + 'VUAsequence/VUA_seq_formatted_val.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert (len(pos_seq) == len(label_seq))
        assert (len(line[2].split()) == len(pos_seq))
        raw_val_vua.append([line[2], label_seq, pos_seq])
        pos_set.update(pos_seq)

# embed the pos tags
pos2idx, idx2pos = get_pos2idx_idx2pos(pos_set)

for i in range(len(raw_train_vua)):
    raw_train_vua[i][2] = index_sequence(pos2idx, raw_train_vua[i][2])
for i in range(len(raw_val_vua)):
    raw_val_vua[i][2] = index_sequence(pos2idx, raw_val_vua[i][2])
print('size of training set, validation set: ', len(raw_train_vua), len(raw_val_vua))

size of training set, validation set:  6323 1550


In [20]:
raw_train_vua[:2]

[["Ca n't fail to be entertaining .",
  [0, 0, 0, 0, 0, 0, 0],
  [8, 9, 8, 14, 8, 13, 0]],
 ['How much was he going to tell her ?',
  [0, 0, 0, 0, 0, 0, 0, 0, 0],
  [9, 13, 8, 1, 8, 14, 8, 1, 0]]]

In [21]:
"""
2. Data preparation
"""
'''
2. 1
get vocabulary and glove embeddings in raw dataset 
'''
# vocab is a set of words
vocab = get_vocab(raw_train_vua)
# two dictionaries. <PAD>: 0, <UNK>: 1
word2idx, idx2word = get_word2idx_idx2word(vocab)
# glove_embeddings a nn.Embeddings
glove_embeddings = get_embedding_matrix(glove_dir + 'glove.840B.300d.txt',word2idx, idx2word, normalization=False)
# elmo_embeddings
elmos_train_vua = h5py.File(elmo_dir + 'VUA_train.hdf5', 'r')
elmos_val_vua = h5py.File(elmo_dir + 'VUA_val.hdf5', 'r')
# no suffix embeddings for sequence labeling
suffix_embeddings = None

vocab size:  13843


100%|██████████| 2196017/2196017 [00:54<00:00, 40492.40it/s]


Number of pre-trained word vectors loaded:  13404
Embeddings mean:  0.0005707233212888241
Embeddings stdev:  0.3729434907436371


In [22]:
print(len(vocab))
glove_embeddings.weight.shape
# 300d embeddings for the 13843 words in the vocab

13843


torch.Size([13845, 300])

In [23]:
'''
2. 2
embed the datasets
'''
# raw_train_vua: sentence, label_seq, pos_seq
# embedded_train_vua: embedded_sentence, pos, labels
embedded_train_vua = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                      glove_embeddings, elmos_train_vua, suffix_embeddings),
                       example[2], example[1]]
                      for example in raw_train_vua]
embedded_val_vua = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                    glove_embeddings, elmos_val_vua, suffix_embeddings),
                     example[2], example[1]]
                    for example in raw_val_vua]

In [24]:
# embedded_train_vua is a list of lists -- one list per sentence
# each sentence list contains 
#     an array of embeddings (seq_length x embedding_dim)
#     list of pos tag ids
#     list of labels
print(len(embedded_train_vua))
print(len(embedded_train_vua[0]))
print(embedded_train_vua[0][0].shape)
print(embedded_train_vua[0][1])
print(embedded_train_vua[0][2])

6323
3
(7, 1324)
[8, 9, 8, 14, 8, 13, 0]
[0, 0, 0, 0, 0, 0, 0]


In [25]:
'''
2. 3
set up Dataloader for batching
'''
# Separate the input (embedded_sequence) and labels in the indexed train sets.
# embedded_train_vua: embedded_sentence, pos, labels
train_dataset_vua = TextDataset([example[0] for example in embedded_train_vua],
                                [example[1] for example in embedded_train_vua],
                                [example[2] for example in embedded_train_vua])
val_dataset_vua = TextDataset([example[0] for example in embedded_val_vua],
                              [example[1] for example in embedded_val_vua],
                              [example[2] for example in embedded_val_vua])

# Data-related hyperparameters
batch_size = 64
# Set up a DataLoader for the training, validation, and test dataset
train_dataloader_vua = DataLoader(dataset=train_dataset_vua, batch_size=batch_size, shuffle=True,
                              collate_fn=TextDataset.collate_fn)
val_dataloader_vua = DataLoader(dataset=val_dataset_vua, batch_size=batch_size,
                            collate_fn=TextDataset.collate_fn)

In [27]:
"""
3. Model training
"""
'''
3. 1 
set up model, loss criterion, optimizer
'''
# Instantiate the model
# embedding_dim = glove + elmo + suffix indicator
# dropout1: dropout on input to RNN
# dropout2: dropout in RNN; would be used if num_layers!=1
# dropout3: dropout on hidden state of RNN to linear layer
RNNseq_model = RNNSequenceModel(num_classes=2, embedding_dim=300 + 1024, hidden_size=300, num_layers=1, bidir=True,
                                dropout1=0.5, dropout2=0, dropout3=0.1)
# Move the model to the GPU if available
if using_GPU:
    RNNseq_model = RNNseq_model.cuda()
# Set up criterion for calculating loss
loss_criterion = nn.NLLLoss()
# Set up an optimizer for updating the parameters of the rnn_clf
rnn_optimizer = optim.Adam(RNNseq_model.parameters(), lr=0.005)
# Number of epochs (passes through the dataset) to train the model for.
num_epochs = 10

In [28]:
'''
3. 2
train model
'''
train_loss = []
val_loss = []
performance_matrix = None
val_f1s = []
train_f1s = []
# A counter for the number of gradient updates
num_iter = 0
comparable = []
for epoch in range(num_epochs):
    print("Starting epoch {}".format(epoch + 1))
    for (__, example_text, example_lengths, labels) in train_dataloader_vua:
        example_text = Variable(example_text)
        example_lengths = Variable(example_lengths)
        labels = Variable(labels)
        if using_GPU:
            example_text = example_text.cuda()
            example_lengths = example_lengths.cuda()
            labels = labels.cuda()
        # predicted shape: (batch_size, seq_len, 2)
        predicted = RNNseq_model(example_text, example_lengths)
        batch_loss = loss_criterion(predicted.view(-1, 2), labels.view(-1))
        rnn_optimizer.zero_grad()
        batch_loss.backward()
        rnn_optimizer.step()
        num_iter += 1
        # Calculate validation and training set loss and accuracy every 200 gradient updates
        if num_iter % 200 == 0:
            avg_eval_loss, performance_matrix = evaluate(idx2pos, val_dataloader_vua, RNNseq_model,
                                                         loss_criterion, using_GPU)
            val_loss.append(avg_eval_loss)
            val_f1s.append(performance_matrix[:, 2])
            print("Iteration {}. Validation Loss {}.".format(num_iter, avg_eval_loss))
#             avg_eval_loss, performance_matrix = evaluate(idx2pos, train_dataloader_vua, RNNseq_model,
#                                                          loss_criterion, using_GPU)
#             train_loss.append(avg_eval_loss)
#             train_f1s.append(performance_matrix[:, 2])
#             print("Iteration {}. Training Loss {}.".format(num_iter, avg_eval_loss))

Starting epoch 1
Starting epoch 2
Starting epoch 3


/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:218: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_text = Variable(eval_text, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:219: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_lengths = Variable(eval_lengths, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:220: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_labels = Variable(eval_labels, volatile=True)


------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  PUNCT 75.0 60.0 66.66666666666667 99.92217898832685
PRFA performance for  PRON nan 0.0 nan 99.68609865470852
PRFA performance for  INTJ nan 0.0 nan 98.74213836477988
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  NOUN 80.9090909090909 38.44492440604752 52.12298682284041 90.46508237352384
PRFA performance for  PROPN 100.0 9.090909090909092 16.666666666666668 98.93276414087514
PRFA performance for  DET 84.30656934306569 92.4 88.16793893129771 98.2490821801751
PRFA performance for  X nan nan nan 100.0
PRFA performance for  VERB 66.33744855967078 60.149253731343286 63.09197651663404 86.8148769574944
PRFA performance for  ADV 61.458333333333336 38.311688311688314 47.2 94.07806191117093
PRFA performance for  CCONJ nan nan nan 100.0
PRFA performance for  NUM nan 0.0 nan 99.77272727272727
PRFA performance for  ADP 81.67883211678833 83.56982823002241 82.61351052048727 89.66198419666374
P

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:351: RuntimeWarning: invalid value encountered in double_scalars
  precision = 100 * grid[1, 1] / np.sum(grid[1])
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:352: RuntimeWarning: invalid value encountered in double_scalars
  recall = 100 * grid[1, 1] / np.sum(grid[:, 1])


Starting epoch 4
Starting epoch 5
------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  PUNCT 75.0 60.0 66.66666666666667 99.92217898832685
PRFA performance for  PRON 0.0 0.0 nan 99.64125560538116
PRFA performance for  INTJ nan 0.0 nan 98.74213836477988
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  NOUN 68.81188118811882 60.043196544276455 64.12918108419838 90.93162268552267
PRFA performance for  PROPN 66.66666666666667 18.181818181818183 28.57142857142857 98.93276414087514
PRFA performance for  DET 85.97785977859779 93.2 89.44337811900192 98.44676645015532
PRFA performance for  X nan nan nan 100.0
PRFA performance for  VERB 64.07307171853857 70.67164179104478 67.21078779276084 87.08053691275168
PRFA performance for  ADV 63.63636363636363 50.0 56.0 94.57155675190668
PRFA performance for  CCONJ 0.0 nan nan 99.92695398100804
PRFA performance for  NUM nan 0.0 nan 99.77272727272727
PRFA performance for  ADP 80.0701754385965 85.2128454

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:353: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2 * precision * recall / (precision + recall)


Starting epoch 6
Starting epoch 7
------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  PUNCT 80.0 80.0 80.0 99.94811932555123
PRFA performance for  PRON 0.0 0.0 nan 99.64125560538116
PRFA performance for  INTJ nan 0.0 nan 98.74213836477988
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  NOUN 75.18573551263002 54.64362850971922 63.2895559724828 91.44190115177139
PRFA performance for  PROPN 71.42857142857143 22.727272727272727 34.48275862068965 98.98612593383137
PRFA performance for  DET 88.14229249011858 89.2 88.66799204771371 98.39028523016097
PRFA performance for  X nan nan nan 100.0
PRFA performance for  VERB 64.92434662998625 70.44776119402985 67.57337151037937 87.33221476510067
PRFA performance for  ADV 63.28125 52.5974025974026 57.4468085106383 94.6164199192463
PRFA performance for  CCONJ nan nan nan 100.0
PRFA performance for  NUM nan 0.0 nan 99.77272727272727
PRFA performance for  ADP 81.49466192170819 85.51157580283794 83.4

In [29]:
"""
for additional training
"""
rnn_optimizer = optim.Adam(RNNseq_model.parameters(), lr=0.0001)
for epoch in range(10):
    print("Starting epoch {}".format(epoch + 1))
    for (__, example_text, example_lengths, labels) in train_dataloader_vua:
        example_text = Variable(example_text)
        example_lengths = Variable(example_lengths)
        labels = Variable(labels)
        if using_GPU:
            example_text = example_text.cuda()
            example_lengths = example_lengths.cuda()
            labels = labels.cuda()
        # predicted shape: (batch_size, seq_len, 2)
        predicted = RNNseq_model(example_text, example_lengths)
        batch_loss = loss_criterion(predicted.view(-1, 2), labels.view(-1))
        rnn_optimizer.zero_grad()
        batch_loss.backward()
        rnn_optimizer.step()
        num_iter += 1
        # Calculate validation and training set loss and accuracy every 200 gradient updates
        if num_iter % 200 == 0:
            avg_eval_loss, performance_matrix = evaluate(idx2pos, val_dataloader_vua, RNNseq_model,
                                                         loss_criterion, using_GPU)
            val_loss.append(avg_eval_loss)
            val_f1s.append(performance_matrix[:, 2])
            print("Iteration {}. Validation Loss {}.".format(num_iter, avg_eval_loss))

#             avg_eval_loss, performance_matrix = evaluate(idx2pos, train_dataloader_vua, RNNseq_model,
#                                                          loss_criterion, using_GPU)
#             train_loss.append(avg_eval_loss)
#             train_f1s.append(performance_matrix[:, 2])
#             print("Iteration {}. Training Loss {}.".format(num_iter, avg_eval_loss))
#             comparable.append(get_performance())

print("Training done!")

Starting epoch 1


/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:218: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_text = Variable(eval_text, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:219: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_lengths = Variable(eval_lengths, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:220: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_labels = Variable(eval_labels, volatile=True)


------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  PUNCT 80.0 80.0 80.0 99.94811932555123
PRFA performance for  PRON 0.0 0.0 nan 99.64125560538116
PRFA performance for  INTJ 100.0 50.0 66.66666666666667 99.37106918238993
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  NOUN 71.89119170984456 59.93520518358531 65.37102473498233 91.42732176702143
PRFA performance for  PROPN 54.54545454545455 27.272727272727273 36.36363636363637 98.87940234791888
PRFA performance for  DET 88.52459016393442 86.4 87.4493927125506 98.2490821801751
PRFA performance for  X nan nan nan 100.0
PRFA performance for  VERB 65.98984771573605 67.91044776119404 66.93637366678928 87.43008948545861
PRFA performance for  ADV 60.15625 50.0 54.60992907801418 94.25751458052939
PRFA performance for  CCONJ 0.0 nan nan 99.92695398100804
PRFA performance for  NUM nan 0.0 nan 99.77272727272727
PRFA performance for  ADP 84.35171385991057 84.54070201643017 84.44610220067139 90

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:353: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2 * precision * recall / (precision + recall)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:351: RuntimeWarning: invalid value encountered in double_scalars
  precision = 100 * grid[1, 1] / np.sum(grid[1])
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:352: RuntimeWarning: invalid value encountered in double_scalars
  recall = 100 * grid[1, 1] / np.sum(grid[:, 1])


Starting epoch 2
Starting epoch 3
------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  PUNCT 80.0 80.0 80.0 99.94811932555123
PRFA performance for  PRON 0.0 0.0 nan 99.64125560538116
PRFA performance for  INTJ 100.0 50.0 66.66666666666667 99.37106918238993
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  NOUN 69.92753623188406 62.52699784017279 66.02052451539339 91.31068668902172
PRFA performance for  PROPN 58.333333333333336 31.818181818181817 41.17647058823529 98.93276414087514
PRFA performance for  DET 87.73946360153256 91.6 89.62818003913895 98.50324767014968
PRFA performance for  X nan nan nan 100.0
PRFA performance for  VERB 64.31893687707641 72.23880597014926 68.04920913884008 87.29026845637584
PRFA performance for  ADV 61.19402985074627 53.246753246753244 56.944444444444436 94.43696724988784
PRFA performance for  CCONJ 0.0 nan nan 99.92695398100804
PRFA performance for  NUM nan 0.0 nan 99.77272727272727
PRFA performance for 

In [54]:
"""
test on genres by POS tags
"""
print("**********************************************************")
print("Evalutation on test set: ")

raw_test_vua = []
with open(data_dir + 'VUAsequence/VUA_seq_formatted_test.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        # txt_id	sen_ix	sentence	label_seq	pos_seq	labeled_sentence	genre
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert(len(pos_seq) == len(label_seq))
        assert(len(line[2].split()) == len(pos_seq))
        raw_test_vua.append([line[2], label_seq, pos_seq])
print('number of examples(sentences) for test_set ', len(raw_test_vua))

for i in range(len(raw_test_vua)):
    raw_test_vua[i][2] = index_sequence(pos2idx, raw_test_vua[i][2])

elmos_test_vua = h5py.File(elmo_dir + 'VUA_test.hdf5', 'r')
# raw_train_vua: sentence, label_seq, pos_seq
# embedded_train_vua: embedded_sentence, pos, labels
embedded_test_vua = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                      glove_embeddings, elmos_test_vua, suffix_embeddings),
                       example[2], example[1]]
                      for example in raw_test_vua]

# Separate the input (embedded_sequence) and labels in the indexed train sets.
# embedded_train_vua: embedded_sentence, pos, labels
test_dataset_vua = TextDataset([example[0] for example in embedded_test_vua],
                              [example[1] for example in embedded_test_vua],
                              [example[2] for example in embedded_test_vua])

# Set up a DataLoader for the test dataset
test_dataloader_vua = DataLoader(dataset=test_dataset_vua, batch_size=batch_size,
                              collate_fn=TextDataset.collate_fn)

print("Tagging model performance on VUA test set by POS tags: regardless of genres")
avg_eval_loss, pos_performance_matrix = evaluate(idx2pos, test_dataloader_vua, RNNseq_model, loss_criterion, using_GPU)

**********************************************************
Evalutation on test set: 
number of examples(sentences) for test_set  2694
Tagging model performance on VUA test set by POS tags: regardless of genres


/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:218: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_text = Variable(eval_text, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:219: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_lengths = Variable(eval_lengths, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:220: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_labels = Variable(eval_labels, volatile=True)


------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  PUNCT 50.0 33.333333333333336 40.0 99.94099134539732
PRFA performance for  PRON nan 0.0 nan 99.84829329962074
PRFA performance for  INTJ 100.0 50.0 66.66666666666667 99.74874371859296
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  NOUN 68.02906448683015 58.01704105344694 62.625418060200666 89.59012575687005
PRFA performance for  PROPN 50.0 12.903225806451612 20.51282051282051 98.37270341207349
PRFA performance for  DET 89.61625282167043 94.74940334128878 92.11136890951276 98.34871296745993
PRFA performance for  X nan nan nan 100.0
PRFA performance for  VERB 68.26625386996903 70.11128775834658 69.17647058823529 88.05713128038897
PRFA performance for  ADV 67.6056338028169 59.01639344262295 63.01969365426695 95.01915708812261
PRFA performance for  CCONJ nan nan nan 100.0
PRFA performance for  NUM nan nan nan 100.0
PRFA performance for  ADP 88.08259587020649 89.40119760479043 88.736

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:351: RuntimeWarning: invalid value encountered in double_scalars
  precision = 100 * grid[1, 1] / np.sum(grid[1])
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:352: RuntimeWarning: invalid value encountered in double_scalars
  recall = 100 * grid[1, 1] / np.sum(grid[:, 1])


In [58]:
print("Breakdown of performance on the VUA sequence labeling test set by POS tags: ")
pd.DataFrame(pos_performance_matrix[[8,4,12,13,14],:], columns = ["Pr", "Re","F1","Acc"],
             index=["VERB","NOUN","ADP","ADJ","PART"])

Breakdown of performance on the VUA sequence labeling test set by POS tags: 


,Pr,Re,F1,Acc
VERB,68.266254,70.111288,69.176471,88.057131
NOUN,68.029064,58.017041,62.625418,89.590126
ADP,88.082596,89.401198,88.736999,92.849057
ADJ,61.464355,58.964880,60.188679,89.356873
PART,56.774194,59.060403,57.894737,91.250854


In [31]:
seq_test_pred = write_predictions(raw_test_vua, test_dataloader_vua, RNNseq_model, using_GPU, data_dir + 'VUAsequence/VUA_seq_formatted_test.csv')

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:304: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_text = Variable(eval_text, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:305: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_lengths = Variable(eval_lengths, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:306: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_labels = Variable(eval_labels, volatile=True)


In [32]:
seq_test_pred[0:2]

[['txt_id',
  'sen_ix',
  'sentence',
  'label_seq',
  'pos_seq',
  'labeled_sentence',
  'genre',
  'prediction'],
 ['a3m-fragment02',
  '45',
  'Design : Crossed lines over the toytown tram : City transport could soon be back on the right track , says Jonathan Glancey',
  '[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0]',
  "['NOUN', 'PUNCT', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'NOUN', 'NOUN', 'VERB', 'ADV', 'VERB', 'ADV', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'VERB', 'PROPN', 'PROPN']",
  'Design : M_Crossed M_lines M_over the toytown tram : City transport could soon be M_back M_on the right M_track , says Jonathan Glancey',
  'news',
  [tensor(0, device='cuda:0'),
   tensor(0, device='cuda:0'),
   tensor(1, device='cuda:0'),
   tensor(1, device='cuda:0'),
   tensor(0, device='cuda:0'),
   tensor(0, device='cuda:0'),
   tensor(0, device='cuda:0'),
   tensor(0, device='cuda:0'),
   tensor(0, device='cuda:0'),
   tensor(0, device='cuda:0'),
   te

In [33]:
get_performance_VUAverb_test(data_dir,seq_test_pred)

Tagging model performance on test-verb: genre
news Precision, Recall, F1, Accuracy:  73.51351351351352 72.98747763864043 73.24955116696589 75.7328990228013
fiction Precision, Recall, F1, Accuracy:  56.56565656565657 61.53846153846154 58.94736842105264 83.10469314079423
academic Precision, Recall, F1, Accuracy:  73.42143906020559 78.36990595611286 75.81501137225172 74.66243050039714
conversation Precision, Recall, F1, Accuracy:  56.22641509433962 51.202749140893474 53.59712230215828 87.10644677661169
Tagging model performance on test-verb: regardless of genre
Precision, Recall, F1, Accuracy:  68.13125695216908 69.56274843838727 68.8395616746277 81.116975991827


array([64.93175606, 66.02464857, 65.40226332, 80.15161736])

In [35]:
#macro-averaged F1 score across four genres
maF1 = (73.2495 + 58.9473 + 75.8150 + 53.5971) / 4
maF1

65.402225

In [34]:
get_performance_VUA_test(data_dir,seq_test_pred)

Tagging model performance on test-sequence: genre
news Precision, Recall, F1, Accuracy:  77.24056603773585 69.97863247863248 73.4304932735426 92.3076923076923
fiction Precision, Recall, F1, Accuracy:  66.57088122605364 68.54043392504931 67.54130223517978 93.9084442823272
academic Precision, Recall, F1, Accuracy:  79.60582690659811 78.99659863945578 79.30004268032437 92.87550495776716
conversation Precision, Recall, F1, Accuracy:  65.53446553446554 65.27363184079601 65.40378863409771 94.77015825169555
Tagging model performance on test-sequence: regardless of genre
Precision, Recall, F1, Accuracy:  74.38683127572017 72.38507127983341 73.37230069816528 93.46287992027902


array([72.23793493, 70.69732422, 71.41890671, 93.46544995])

In [36]:
import pandas as pd
gao_scores_verb = [68.2, 71.3, 69.7, 81.4, 66.4]
our_scores_verb = [68.0, 68.3, 68.2, 80.9, 65.4]
our_scores_verb = [round(score,1) for score in our_scores_verb]
all_scores_verb = [gao_scores_verb, our_scores_verb]
all_scores_verb_df = pd.DataFrame(all_scores_verb, columns= ['P', 'R', 'F1', 'Acc', 'MaF1'], index=['Gao et al', 'US'])
print("VUA seq model: classification task\n")
all_scores_verb_df

VUA seq model: classification task



,P,R,F1,Acc,MaF1
Gao et al,68.2,71.3,69.7,81.4,66.4
US,68.0,68.3,68.2,80.9,65.4


In [43]:
!g

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
	modified:   notebooks/sequence/replicate_vua_seq_model_for_classification.ipynb

no changes added to commit
